In [2]:
# Modules: Email_Parser and PDF_Converter
from email import policy
from email.parser import BytesParser
import glob
import pdfkit
import os
from bs4 import BeautifulSoup
import datetime 
from pathlib import Path

# Modules: Registration List and Folder Creator
import os 
import pdfquery
import pandas as pd

# Modules: Folder Creator 
import os
import pandas as pd
import numpy as np
import shutil
import pathlib
import xlwings as xw


In [3]:
# extract_file_url() is a function that takes in a list of .eml files.
# It parses the .eml file, extracts the URL from the .eml file
# and returns a dictionary with the file name as the key and the URL as the value

def extract_file_url(student_file):
    student_dict = dict()

    # Iterate through the directory of .eml files
    for student in student_file:

        with open(student, 'rb') as fp:  
            # Parse the email, get the link to the registration form.
            msg = BytesParser(policy=policy.default).parse(fp)
            text = msg.get_payload(decode=True)
            soup = BeautifulSoup(text, 'html.parser')
            registration_link = soup.find_all('a')[1].get('href')
            student_dict[student] = [registration_link]

            # Get the time the email was submitted, format the time. 
            time_submitted = msg['Received'].split(';')[1].strip()
            time_submitted = datetime.datetime.strptime(time_submitted, '%a, %d %b %Y %H:%M:%S %z')
            time_submitted = time_submitted.strftime('%m-%d-%Y %H:%M:%S')
            student_dict[student].append(time_submitted)

    return student_dict

name = extract_file_url(glob.glob('*.eml'))

In [4]:
# get_pdf() is a function that takes in a dictionary of student names and their registration form URL.
# It uses pdfkit to convert the URL to a PDF and saves the PDF in the same directory as the script.
# TODO: Remove the .eml files after the PDFs have been generated.

def get_pdf(student_dict):
    
    count = 0
    
    options = {'orientation': 'Landscape'}

    for key, value in student_dict.items():
        out_file = key + '.pdf'
        pdfkit.from_url(value, out_file,options = options)
        count +=1
        
    print('Done!')
    print(f'A total of {count} pdfs were generated.')

get_pdf(name)

Done!
A total of 9 pdfs were generated.


In [5]:
def move_pdf():

    os.mkdir("Generated PDFs")
    for file in glob.glob("*.pdf"):
        os.rename(file, "Generated PDFs/" + file)
    
    print('Done!')
    print('All PDFs have been moved to the Generated PDFs folder.')


move_pdf()


Done!
All PDFs have been moved to the Generated PDFs folder.


At this point the following has been created: 

1) PDFs of Student Applications have been generated and saved locally. 
2) Information related to applications has been scraped loaded into xlsx file for further processing. 


In [6]:
def clean_dataframe(student_dictionary):
    df = pd.DataFrame.from_dict(student_dictionary, orient ='index')
    df = df.reset_index().rename(columns={'index':'Student Name', 0:'URL', 1:'Time Submitted'})
    df = df.drop(columns =['URL'])
    df['Student Name'] = df['Student Name'].str.replace('.eml', '', regex=True)

    return df 

df = clean_dataframe(name)

,Student Name,Time Submitted
0,Jose Rivera 2,02-05-2023 02:10:17
1,Jacob Gonzalez,02-07-2023 03:40:26
2,Paola Garcia 2,02-14-2023 17:40:22
3,Keyuiona Douglas,02-18-2023 20:10:19
4,Olivia Garcia,01-31-2023 20:40:17
5,Varsha Mangtani,01-31-2023 23:00:23
6,Lizbeth Jaime,01-31-2023 22:30:20
7,Mia Garcia,03-28-2023 04:10:50
8,Maxwell Walters,02-01-2023 07:00:19
